In [ ]:
import os
import csv 
import json
import math
import requests
import numpy as np
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup
from collections import Counter
from datetime import datetime as DT
from datetime import timezone

In [2]:
# initialize the path variables of your own piazza dataset
COURSE = "sample_course"
SEMESTER = "sample_semester"
semesters = {"sample_course": ["sample_semester"]}

def set_paths(course=COURSE, semester=SEMESTER):
    BASE = f'{course}_data/'
    PATH = f"{course}_data/Piazza_Data/Piazza_{semester}/"
    IMG_PATH = f"{course}_data/Processed_Data/Piazza_{semester}/images/"
    PROC_PATH = f"{course}_data/Processed_Data/Piazza_{semester}/"
        # Create directories if they don't exist
    if not os.path.exists(IMG_PATH):
        os.makedirs(IMG_PATH)
    return BASE, PATH, IMG_PATH, PROC_PATH

BASE, PATH, IMG_PATH, PROC_PATH = set_paths()

def get_part_csv(course=COURSE, part='QA', path=BASE):
    return f'{path}{course}_{part}_nofollowup_preOCR.csv'

# Helper functions

In [3]:
def write_to_json(directory, file_name, json_obj_lst):
    p = Path(directory)
    p.mkdir(parents=True, exist_ok=True)
    file = f"{directory}/{file_name}"
    if not os.path.exists(file):  # create file not exist and write empty list
        with open(file, 'w+') as f:
            json.dump([], f)
    with open(file) as f:
        json_list = json.load(f)
        json_list += json_obj_lst

    with open(file, 'w') as f:
        json.dump(json_list, f, separators=(',', ': '), indent=4)
    return

def to_json(data):
    if data is None or isinstance(data, (bool, int, str)): return data
    if isinstance(data, np.integer):
        return int(data)
    if isinstance(data, list):
        return [to_json(v) for v in data]
    if isinstance(data, dict):
        return {to_json(key): to_json(value) for key, value in data.items()}
    else:
        print("other type", type(data), data)

In [4]:
# zulutime = "2021-08-31T20:06:32.011Z", 2019-07-09T05:46:25.557Z
def zulu_to_utc(zulutime):
    dt = DT.fromisoformat(zulutime.replace('Z', '+00:00'))
    # utctime => "2021-08-31 20:06:32 UTC"
    return dt.strftime("%Y-%m-%d %H:%M:%S UTC")

def zulu_near_utc(zulutime):
    # return utc0, utc1, utc2 that's one second before, at, and after zulutime
    dt = DT.fromisoformat(zulutime.replace('Z', '+00:00'))
    utc_format = "%Y-%m-%d %H:%M:%S UTC"
    utc1 = dt.strftime(utc_format)
    utc2 = (dt + pd.Timedelta(seconds=1)).strftime(utc_format)
    utc0 = (dt - pd.Timedelta(seconds=1)).strftime(utc_format)
    return utc0, utc1, utc2

# define is_close function to check if two timestamps are close enough
def is_close(dt1, dt2):
    diff = dt1 - dt2
    return abs(diff.total_seconds()) < 1

# zulu = "2015-10-20T00:15:52.992Z" is_close utc = "2015-10-20 00:15:53 UTC"
def zulu_vs_utc(zulutime, utc):
    dt_zulu = DT.fromisoformat(zulutime.replace('Z', '+00:00'))
    # print(dt.utcfromtimestamp(dt.timestamp()))
    dt_utc = DT.strptime(utc, '%Y-%m-%d %H:%M:%S %Z').replace(tzinfo=timezone.utc)
    # Note Excluded time components are truncated, not rounded.
    return is_close(dt_zulu, dt_utc)

def find_match_time(zulu, row):
    return zulu_vs_utc(zulu, row['Created At'])


In [5]:
# TODO: there are href tags in the html that isn't processed yet
# with .png or .jpg as the ending of href, e.g., url = f"https://piazza.com/{a.get('href')}"
# there can be <md> tags and images in markdown too, <a href in markdown2.markdown(tag.text))
def str_html(html):
    # turn br tag into \n
    soup = BeautifulSoup(html, 'html.parser')
    for br in soup.find_all("br"): #["br", "md"] print("adding br.text?", br.text)
        br.replace_with("\n" + br.text)
    return soup.get_text() 

def request_html(url):
    try:
        r = requests.get(url, timeout=10)
    except Exception as e:
        # print("Exception occur while requesting", e)
        return False, None
    if r.status_code != 200: 
        print("Error requesting", r.status_code, url, r.content)
        return False, None
    return True, r.content

def extract_img_from_html(html_text, postid, post_part, postnum, semester, course=COURSE, IMG_PATH=IMG_PATH):
    # output img file already with ID, src_url (unique key), url, postid, idx, and then ocr
    soup = BeautifulSoup(html_text, 'html.parser')
    images = []
    # IMG_PATH = f"{course}_data/Processed_Data/Piazza_{semester}/images/"
    for (index, img) in enumerate(soup.find_all('img')):
        src_url = img.get('src')
        url = src_url
        success_src, content = request_html(src_url)
        if not success_src:
            url = f"https://piazza.com{src_url}"
            success, content = request_html(url)
            if not success:
                print("neither urls worked", src_url, url)
                continue

        img_name = f"{postid}_{index}.png"
        # print(postnum, img_name, len(content), post_part)
        with open(f"{IMG_PATH}{img_name}", 'wb') as f:
            f.write(content)
        images.append({"img_id": img_name, "semester": semester, "postnum": postnum, "post_part": post_part, "src_url": src_url, "success_url": url, "ocr_text": None, "ocr_valid": None})
    return images

# TODO if ocr=True, read from images.json in IMG_PATH
def process_html_text(html_text, postid, ocr=False, IMG_PATH=None):
    if not ocr: return str_html(html_text)
    # with the postid, find the images urls and the ocr_text
    # then match src_url with html_text using soup.find_all('img', {'src': src_url}) and replace with <pre>ocr_text</pre>
    # return the str_html of new html_text

# Read headers & first entry of different files

In [6]:
# "class_content_flat.json"
def get_class_content_flat(path):
    with open(f"{path}/class_content_flat.json") as f:
        class_content_flat = json.load(f)
    print("class_content_flat.json", len(class_content_flat))
    return class_content_flat

# "contributions.csv"
def get_contributions(path):
    with open(f"{path}/contributions.csv") as f:
        contributions = pd.read_csv(f)
    print(path, "contributions.csv", len(contributions))
    labels = set()
    for folders in set(contributions["Folders"]):
        if type(folders) != str: 
            print(folders) # folders can also be nan
            continue
        labels.update(folders.split("; "))
    print("Folders", len(labels), labels)
    return contributions

# Anonymize data & remove identifiable info
Note that if your dataset is not pre-anonymized, consider remove at least these fields & replace with anonymize user_id. Although the dataset we used is already anonymized, some reference code is provided for your convenience. 

Note that we can also infer teh roles of student vs. instructors (those who answered questions)
- `users.json`
  - 'name': 
  - 'email': 
  - 'lti_ids': []
- `contributionw.csv`
  - Name	Email -> replace with user_id

In [7]:
# proxy to instructor: who answered question 
def get_proxy_to_instructor(class_content_flat):
    instructors = set()
    for post in class_content_flat:
        if post["type"] == "i_answer":
            instructors.update(post["editors"])
    return instructors

# after added the role inference, return a set of instructors from users.json "role" == "instructor"
def get_instructors():
    with open(f"{PATH}users.json", 'r') as f:
        users = json.load(f)
        instructors = set([user["user_id"] for user in users if user["role"] == "instructor"])
    print("instructors", len(instructors), instructors)
    return instructors

def remove_config(path):
    # remove file "config.json" from PATH
    if os.path.exists(f"{path}/config.json"):
        os.remove(f"{path}/config.json")

def anonymize_users(instructors, path, proc_path):
    name_to_id, email_to_id = {}, {"Not currently enrolled": "NotEnrolled", "Anon": "Anonymous"}
    print("# instructors: ", len(instructors)) 
    if os.path.exists(f"{path}/users.json"):
        with open(f"{path}/users.json") as f:
            users = json.load(f)
            print("# Piazza users in total:", len(users))
            anonymized_users = []
            for user in users:
                email_to_id[user["email"]] = user["user_id"]
                name_to_id[user["name"]] = user["user_id"]
                user["role"] = 'instructor' if user["user_id"] in instructors else 'student'
                anonymized_user = {"user_id": user["user_id"], "role": user["role"], "days": user["days"], "posts": user["posts"], "asks": user["asks"], "answers": user["answers"], "views": user["views"]}
                anonymized_users.append(anonymized_user)
        return anonymized_users, name_to_id, email_to_id

# replace the Name and Email column of contributions.csv with user_id
def replace_name_email_with_id(contributions, email_to_id):
    if "Email" not in contributions.columns and "UserID" in contributions.columns: return contributions
    contributions["UserID"] = contributions["Email"].apply(lambda x: email_to_id[x])
    # select a subset of columns to write to file
    contributions = contributions[['Anonymous', 'Post Number', 'Folders', 'Created At', 'Submission', 'Submission HTML Removed', 'Subject', 'Part of Post', 'Endorsed by Instructor', 'UserID']]
    return contributions

def anonymize_all_semesters(sem_list, course=COURSE):
    # for course, sems in semesters.items():
    for sem in sem_list:
        PATH, IMG_PATH, PROC_PATH = set_paths(course, sem)

        remove_config(PATH)
        contributions = get_contributions(PATH)
        class_content_flat = get_class_content_flat(PATH)
        instructors = get_proxy_to_instructor(class_content_flat)
        anonymized_users, name_to_id, email_to_id = anonymize_users(instructors, PATH, PROC_PATH)
        contributions = replace_name_email_with_id(contributions, email_to_id)

        # re-write
        with open(f"{PATH}/users.json", 'w') as f: json.dump(anonymized_users, f, indent=4)
        # don't save these mappings for the final anonymized dataset
        # with open(f"{PROC_PATH}/name_to_id.json", 'w') as f: json.dump(name_to_id, f, indent=4)
        # with open(f"{PROC_PATH}/email_to_id.json", 'w') as f: json.dump(email_to_id, f, indent=4)
        contributions.to_csv(f"{PATH}/contributions.csv", index=False)

# uncomment this line if your dataset is not anonymized yet
# anonymize_all_semesters([SEMESTER])

# Match class_content_flat.json (final posts) with contributions.csv
- ## class_content_flat.json: the finalized post
	- `'id'`
	- `'type'`:		'note', 'question',' i_answer', 's_answer', 'followup', followup_response'
	- `'created'`: 	Zulu time
	- `'subject'`:	come with the question or note type
	- `'content'`:	HTML text, can be matched with Submission after .strip() and taken account of '' cases
	- `'parent_id'`:	parent thread id, a followup's parent is the original post, a followup_response's parent is the followup, multiple followup_response to the same followup share the same parent_id
	- `'thread_id'`:	original post id, the thread's root, a followup & followup_response's thread is the original post
	- `'views'`
	- `'editors'`:	list of user_id
	- `'score'`:		len(tag_good_arr), # people liked this post
	- `'tag_good_arr'`
	- `'anonimity'`

- ## contributions.csv: all edit history of post
 	* `Anonymous`,
	* `Post Number`:	@piazza post num for reference
	* `Folders`:		list of labels/folders that the post has been bucketed into on piazza, e.g., 
					'exercises', 'hw1', 'hw4', 'hw3', 'hw6', 'hw2', 'quiz5', 'quiz4', 'quiz3', 'hw5', 'other', 'logistics', 'lecture', 'quiz2', 'final', 'quiz1'
	* `Created At`:	UTC time
	* `Submission`:	HTML text, can be NaN with pd
	* `Submission HTML Removed`,
	* `Subject`,
	* `Part of Post`: 'followup', 'reply_to_followup', 
					'started_off_i_answer', 'started_off_note', 'started_off_poll',
					'started_off_question', 'started_off_s_answer', 'updated_i_answer', 'updated_note', 'updated_question', 'updated_s_answer'
	* `Endorsed by Instructor`: True, False, NaN

In [8]:
def nan_to_false(endorsed):
    if isinstance(endorsed, bool): return endorsed 
    else: return False # T/F, make NaN to F too

def get_additional_fields(match):
    postnum = match['Post Number'].values[0] # the post number
    folders = match['Folders'].values[0] # the folders list
    endorsed = match['Endorsed by Instructor'].values[0] # the endorsed label
    utc = match['Created At'].values[0] # use the utc time of the updated content
    # edit = match['Part of Post'].values[0] # the edit label, e.g., 'started_off_s_answer', 'updated_i_answer', 'started_off_i_answer', 'updated_s_answer'
    return utc, postnum, folders, nan_to_false(endorsed)

def check_match_content(final_content, contributions, postnum=None):
    match_content = contributions[contributions["Submission"] == final_content]
    if postnum != None:
        match_content = match_content[match_content["Post Number"] == postnum]
    if len(match_content) == 1: # there's a match in content
        return get_additional_fields(match_content)
    elif len(match_content) > 1: # there's more than one match in content, probably update something else
        # choose the one with later Created At field, but also matching postnum
        match_content = match_content.sort_values(by=['Created At'], ascending=False)
        return get_additional_fields(match_content)
    # else: print("no match in content") 

def get_updated_answer(post_type, post_part, post_threads, postid, post_content, updated_ans, match_time, role='i'):
    assert(post_type == f"{role}_answer")
    if post_part == f'updated_{role}_answer':
        started_off_answer = post_threads[post_threads["Part of Post"] == f'started_off_{role}_answer']
        assert len(started_off_answer) == 1, f"len(started_off_s/i_answer) = {len(started_off_answer)} != 1"
        return process_html_text(started_off_answer["Submission"].values[0], postid, ocr=False)
    else:
        assert(post_part == f'started_off_{role}_answer')
        updated_answer = post_threads[post_threads["Part of Post"] == f'updated_{role}_answer']

        if len(post_threads[post_threads["Part of Post"] == f'updated_{role}_answer']) != 0:
            print("postid", postid, post_type, post_part, role)
            print("post_content", post_content)
            print("updated_ans_clean_content", updated_ans)
            print("updated_answer", updated_answer)
            print("match_time", match_time)
            print("post_threads", post_threads)
            assert False, f"len(updated_{role}_answer) != 0"

def all_edit_history_same(match_content):
    # not necessarily len=2 match_content["Submission"].values[0] == [1] + .strip())
    contents = set(match_content["Submission"].values)
    stripped_contents = set([content.strip() for content in match_content["Submission"].values])
    return len(contents) == 1 or len(stripped_contents) == 1 # all edits are the same


def find_updated(match_time, postnum, postid, post_type, post_content, updated_ans, contributions):
    post_threads = contributions[contributions["Post Number"] == postnum]
    answers_part = ['started_off_i_answer', 'updated_i_answer', 'started_off_s_answer', 'updated_s_answer']
    match_content = post_threads.loc[((post_threads["Submission"] == post_content) | (post_threads["Submission"] == post_content.strip())) & 
                                     (post_threads["Part of Post"].isin(answers_part))]
    # after filter part of post == answer, it's possible that content literally didn't change among edits
    if len(match_content) > 1: # since filter used strip, possible edits like \n\n, so compare both strip too
        assert all_edit_history_same(match_content), \
                f"len(match_content) = {len(match_content)}, {match_content['Submission'].values}"
        # manually only keep the started_off_{}_answer in that case
        return {"updated_answer": None, "started_off_answer": updated_ans}
    elif len(match_content) == 0:
        print("match_time post, why would there be mismatched postnum?", match_time)
        print("post_threads", post_threads)
        print("postid", postid, "postnum", postnum, post_type)
        print("post_content", post_content, "updated_ans:", updated_ans)
        assert False, "len(match_content) == 0"
    
    assert len(match_content) == 1, f"len(match_content) = {len(match_content)}, match_content = {match_content['Submission'].values}"
    post_part = match_content["Part of Post"].values[0]
    role = post_part.split('_')[-2]
    assert role in ['s', 'i']
    init_ans = get_updated_answer(post_type, post_part, post_threads, postid, post_content, updated_ans, match_time, role)
    if init_ans == None: # print("no edits existed")
        return {"updated_answer": None, "started_off_answer": updated_ans}
    else:
        return {"updated_answer": updated_ans, "started_off_answer": init_ans}

In [9]:
# make a dictionary with key being postnum
# value being class_content_flat with additional fields from contributions.csv
def get_all_posts(class_content_flat, contributions):
    all_posts = {}
    for post in class_content_flat:
        if post['type'] == 'poll': continue # skip poll
        final_content = post['content'].strip() if isinstance(post['content'], str) else ""
        # cutomize & parse img links & ocr
        clean_content = process_html_text(final_content, post['id'], ocr=False)
        # find the row in contributions that has value at 'Created At' column == utctime
        # utc = zulu_to_utc(post["created"])
        utc0, utc1, utc2 = zulu_near_utc(post["created"])
        match_time = contributions[ (contributions["Created At"] == utc0) | 
                                    (contributions["Created At"] == utc1) | (contributions["Created At"] == utc2) ]
        # match_time = contributions[contributions.apply(lambda row: find_match_time(post["created"], row), axis=1)]
        if len(match_time) == 1: # there's a match in timestamp (started off time, not edited time)
            # the timestamp is the started off time, not the edited time, so content may be different
            utc, postnum, folders, endorsed = get_additional_fields(match_time)
            # extract the string content in match['Submission'] and compare with post["content"]
            same_time_content = match_time['Submission'].values[0]
            if final_content == "":
                if isinstance(same_time_content, str): # the content is deleted
                    print("Deleted/Resolved empty post, ignore it", match_time.values)
                    continue
                else:
                    assert(math.isnan(same_time_content)) # empty post match, question in subject
                    print(postnum, "empty post match, question in subject")

            elif (isinstance(same_time_content, str) and same_time_content.strip() != final_content):
                # and str_html(final_content) != match_time['Submission HTML Removed'].values[0].strip()): # the content is edited
                # html or html_removed version is the same? 
                # updated post? but updated content may be empty, so postnum can be matched to a wrong one 
                result = check_match_content(post['content'], contributions, postnum)
                if result == None: 
                    print("Can be possible if student not currently enrolled, ignore")
                    # print(post, match_time.values)
                    continue
                else: utc, postnum, folders, endorsed = result
            # else: pass # perfect match
        
        else: # TODO, only use content to check match may still be buggy, postnum may be wrong
            if len(match_time.values) > 1: # multiple same timestamp match
                result = check_match_content(post['content'], contributions)
                if result == None: 
                    print("\nNo content match, is empty/duplicate content? ", len(post['content'])==0, 
                          len(match_time.values), "posts same time", match_time.values)
                    continue
                else: utc, postnum, folders, endorsed = result

            else: # there's no match in timestamp for class_content_flat.json & contributions.csv
                assert len(match_time.values) == 0, f"No match in time? len(match_time.values) = {len(match_time.values)}"
                result = check_match_content(post['content'], contributions)
                if result == None: continue
                else: utc, postnum, folders, endorsed = result
                print("No match in time but found match content", postnum, post['id'], utc, postnum)
                # print('post["created"]->utc', post["created"], zulu_to_utc(post["created"]))
                # print("rematched using content", utc, type(utc))

        addition = {"utc": utc, "postnum": postnum, "folders": folders, "endorsed": endorsed, "clean_content": clean_content}
        # find edited post, started_off_s_answer, updated_i_answer, started_off_i_answer, updated_s_answer
        if post['type'] in ['i_answer', 's_answer']: 
            answer_edit = find_updated(match_time, postnum, post['id'], post['type'], post['content'], clean_content, contributions)
            addition['answer_edit'] = answer_edit
        post.update(addition)
        # add to all_posts and setdefault = [], if exist append
        all_posts.setdefault(postnum, []).append(post)

    return all_posts

# Save Naive QA data without follow ups
- class_content_flat.json: the finalized post ```'id', 'subject', 'parent_id', 'tag_good_arr', 'editors', 'anonimity', 'created', 'views', 'thread_id', 'content', 'score', 'type'```
- contributions.csv: all edit history of post ```Anonymous	Post Number	Folders	Created At	Submission	Submission HTML Removed	Subject	Part of Post	Endorsed by Instructor```

In [14]:
# a csv file of the following columns for each semesters data: {COURSE}_{SEMESTER}_QA_nofollowup.csv
# postnum, subject, question, answer, answer_by, folders, views, utc_q, utc_a, numlikes_q, numlikes_a, endorsed_q, endorsed_a
def extract_qa(posts):
    q_dict, a_dict = {}, {}
    for post in posts:
        if post['type'] == "question": 
            assert(post['score'] == len(post['tag_good_arr']))
            q_dict = {"id_q": post['id'], "subject": str_html(post['subject']), 
                      "question": post['clean_content'], 
                      "html_q": post['content'], "folders": post['folders'], 
                      "views": post['views'], "utc_q": post['utc'], 
                      "numlikes_q": post['score'], "endorsed_q": post['endorsed']}
        
        elif post['type'] in ["i_answer", "s_answer"]:  # a does not have score field
            a_dict.update({post['type']: 
                           {**post['answer_edit'],
                            "id_a": post['id'],
                            "html_a": post['content'], 
                            "answer_by": 'student' if post['type'] == 's_answer' else 'instructor',
                            "utc_a": post['utc'], 
                            "numlikes_a": len(post['tag_good_arr']), "endorsed_a": post['endorsed']}})
        # else: continue # skip other types like note & followup for now
    return q_dict, a_dict

def create_qa(semester, all_posts, QAWriter):
    qa_count, notes = 0, {}
    for postnum, posts in all_posts.items():
        q_dict, a_dict = extract_qa(posts)
        if len(q_dict) == 0 or len(a_dict) == 0: # print("No question or answer found", posts)
            notes[postnum] = posts # add to the notes dict
            continue
        for a in a_dict.values():
            qa_count += 1
            QAWriter.writerow({"semester": semester, "postnum": postnum, **q_dict, **a})
    
    print(f"For {semester}, total #posts = {len(all_posts)}, QA pairs without followups = {qa_count}")
    return notes, {"total_posts": len(all_posts), "total_qas": qa_count}

def create_notes(semester, notes, NoteWriter):
    notes_count = 0
    for postnum, posts in notes.items():
        for post in posts:
            if post['type'] == 'note':
                notes_count += 1
                note = post['clean_content']
                subject = str_html(post['subject'])
                utc = post['utc']
                NoteWriter.writerow({"semester": semester, 'postnum': postnum, 'id': post['id'], 
                'subject': subject, 'utc': utc, 'note': note, 'html': post['content']})
    
    print(f"#note posts = {notes_count}")
    return {"total_notes": notes_count}

def get_raw_materials(course, semester):
    # PATH = set_paths(course, semester)[1]
    print(PATH)
    contributions = get_contributions(PATH)
    class_content_flat = get_class_content_flat(PATH)
    return class_content_flat, contributions

## Save QA data with following format
-  a csv file of the following columns for each semesters data: {COURSE}_{SEMESTER}_QA_nofollowup.csv
``` postnum, subject, question, s_answer, i_answer, folders, views, utc_q, utc_a, numlikes_q, numlikes_a, endorsed_q, endorsed_a```

## Each piazza post is one row
- postnum: can be useful for reference pointing, e.g., 
  - Check @887 to see if it helps you understand the operations and their order
  - You can check the post @118_f1 at lecture 2 thread
- question: `updated_question` can be better than `started_off_question`, as we don't clear about changes in question wording for model to learn, only keep `updated_question`
- answer: {started_off/updated}_{i/s}_answer
  - `X_i_answer` is from `i`nstructor, and `X_s_answer` is from `s`tudent
  - `updated_X_answer` can be naively seen as better than `started_off_X_answer` after edit
- answer_by: instructor/student
- folders: I kept the original folder labels, note that across different semesters, things may be labled slightly different, e.g., `quiz` vs. `quizzes`, `hw1` vs. `hw1p1`, etc. 
- endorsed: T/F endorsed by instructor
- numlikes: tag_good_arr (user_ids), score being #people that liked this post? edited by? viewed by?
- followup: a dictionary of threads {"thread_id": ["", "", ], ...}, some info are not saved in this format, e.g., if an instructor endorsed an followup response, etc.
- images are labeled with [IMG][prepended link] in text => **TODO OCR**

In [17]:
def parse_all_semesters(sem_list, COURSE):
    counts = Counter(total_posts=0, total_qas=0, total_notes=0)
    all_posts, all_notes = dict(), dict()
    with open(f'{BASE}/{COURSE}_QA_nofollowup_preOCR.csv', 'w') as csvfile:
        QAWriter = csv.DictWriter(csvfile, fieldnames=['semester', 'postnum', 'subject', 'question', 
        'started_off_answer', 'updated_answer', 'answer_by', 'folders', 'views', 'html_q', 'html_a',
        'id_q', 'id_a', 'utc_q', 'utc_a', 'numlikes_q', 'numlikes_a', 'endorsed_q', 'endorsed_a'])
        QAWriter.writeheader()
        for semester in sem_list:
            print(f"\n\n#### Processing {COURSE} {semester} \n")
            class_content_flat, contributions = get_raw_materials(COURSE, semester)
            # generate QA data & notes (without followups)
            post_semester = get_all_posts(class_content_flat, contributions)
            notes, update_counts = create_qa(semester, post_semester, QAWriter)
            all_notes[semester] = notes
            # using a Counter dict to count the number of posts, qas, notes for each semester
            counts.update(update_counts)
            post_semester = {f"{semester}_{postnum}": posts for postnum, posts in post_semester.items()}
            all_posts.update(post_semester)

    with open(f'{BASE}/{COURSE}_notes_nofollowup_preOCR.csv', 'w') as notecsv:
        NoteWriter = csv.DictWriter(notecsv, fieldnames=['semester', 'postnum', 'id', 'subject', 'utc', 'note', 'html'])
        NoteWriter.writeheader()
        for semester, notes in all_notes.items():
            update_counts = create_notes(semester, notes, NoteWriter)
            counts.update(update_counts)
    
    with open(f"all_posts_semesters.json", 'w') as f: json.dump(to_json(all_posts), f, indent=4)
    print(f"Out of {len(semesters[COURSE])} semesters of {COURSE}, {counts}")


In [ ]:
# sem_list = semesters[COURSE]
parse_all_semesters([SEMESTER], COURSE)

# load all_posts from previous run saved json
with open(f"all_posts_semesters.json", 'r') as f:
    all_posts = json.load(f)
print(len(all_posts))

# Note that to really loop through all semesters, get_instructors() can't just take the default PATH
all_instructors = {SEMESTER: get_instructors()}



#### Processing sample_course sample_semester 

sample_course_data/Piazza_Data/Piazza_sample_semester/
sample_course_data/Piazza_Data/Piazza_sample_semester/ contributions.csv 15
Folders 3 {'logistics', 'lab5', 'c_project'}
class_content_flat.json 11
For sample_semester, total #posts = 3, QA pairs without followups = 3
#note posts = 0
Out of 1 semesters of sample_course, Counter({'total_posts': 3, 'total_qas': 3, 'total_notes': 0})
3
instructors 3 {'i6q23gcekfm15v', 'gooyf9nn3PO', 'l01f6628pmr4ce'}


In [19]:
all_qas = pd.read_csv(get_part_csv(COURSE, "QA"))
all_notes = pd.read_csv(get_part_csv(COURSE, "notes"))
print("all_qas", len(all_qas), all_qas.columns)
print("all_notes", len(all_notes), all_notes.columns)

all_qas 3 Index(['semester', 'postnum', 'subject', 'question', 'started_off_answer',
       'updated_answer', 'answer_by', 'folders', 'views', 'html_q', 'html_a',
       'id_q', 'id_a', 'utc_q', 'utc_a', 'numlikes_q', 'numlikes_a',
       'endorsed_q', 'endorsed_a'],
      dtype='object')
all_notes 0 Index(['semester', 'postnum', 'id', 'subject', 'utc', 'note', 'html'], dtype='object')


In [20]:
# process {course}_QA_nofollowup_preOCR.csv's column of html, produce images into folders
# process {course}_notes_nofollowup_preOCR.csv's column of html, produce images into folders
# eliminate rows with images, produce new csv files 

def download_images_label_qa(row, course=COURSE):
    html_q, html_a = row["html_q"], row["html_a"]
    imgs_q = extract_img_from_html(html_q, row["id_q"], "question", row["postnum"], row["semester"]) if isinstance(html_q, str) else []
    imgs_a = extract_img_from_html(html_a, row["id_a"], "answer", row["postnum"], row["semester"]) if isinstance(html_a, str) else []
    row["num_images"] = len(imgs_q) + len(imgs_a)
    # if row["num_images"] > 0 and ( row["semester"] == 2022 or (row["semester"] == 2021 and row["postnum"] >= 1548) ):
    if row["num_images"] > 0: write_to_json(f"{course}_data/Processed_Data", "images.json", imgs_q + imgs_a)
    return row

def download_images_label_notes(row, course=COURSE):
    html = row["html"]
    imgs = extract_img_from_html(html, row["id"], "note", row["postnum"], row["semester"]) if isinstance(html, str) else []
    row["num_images"] = len(imgs)
    if row["num_images"] > 0: write_to_json(f"{course}_data/Processed_Data", "images.json", imgs)
    return row

### Downloading images in posts

In [ ]:
all_qas = all_qas.apply(lambda row: download_images_label_qa(row), axis=1)
all_qas.to_csv(f"{BASE}/{COURSE}_QA_nofollowup_preOCR_imgtag.csv", index=False)

all_notes = all_notes.apply(lambda row: download_images_label_notes(row), axis=1)
# all_notes.to_csv(f"{BASE}/{COURSE}_notes_nofollowup_preOCR_imgtag.csv", index=False)
all_qas.tail()

,semester,postnum,subject,question,started_off_answer,updated_answer,answer_by,folders,views,html_q,html_a,id_q,id_a,utc_q,utc_a,numlikes_q,numlikes_a,endorsed_q,endorsed_a,num_images
0,sample_semester,7,coursebook unlock,when i currently try to download the matlab co...,Seems to be unblocked now. Try again?,NaN,instructor,logistics,12,when i currently try to download the matlab co...,Seems to be unblocked now. Try again?,l5lje4qurpswp,l5ljutq8avt6w4,2022-07-14 21:21:59 UTC,2022-07-14 21:34:58 UTC,0,2,False,True,0
1,sample_semester,486,Lab 5.5,When I am testing I had this error.\nHow can I...,"As KW would say:\n\n""we cannot help you to deb...","As KW would say:\n\n""we cannot help you to deb...",instructor,lab5,281,<p>When I am testing I had this error.</p>\n<p...,<p>As KW would say:</p>\n<p></p>\n<p>&#34;we c...,l6x5f4tlltb1ey,l6x7po6chew18,2022-08-17 05:03:48 UTC,2022-08-17 06:37:50 UTC,0,0,False,False,1
2,sample_semester,1477,Can you put an if statement inside another if ...,.,Can you? Yes. Should you? Sometimes. For the p...,NaN,student,c_project,114,<p>.</p>,<p>Can you? Yes. Should you? Sometimes. For th...,l97k23y4ceh6ck,l97zi81rtwt3fb,2022-10-13 21:10:41 UTC,2022-10-14 04:23:07 UTC,0,0,False,False,0


## Format each QA post into json files with entries
```
instruction: post['question']
input: ''
output: post['started_off_answer']
text: 'You're a helpful and knowledgeable teaching assistant in CS1. 
      Answer this question for a novice student. 
      ### Question: {post['question']} ### Answer: {post['started_off_answer']}
```


In [24]:
# TODO: some filtering mechanisms errors/num_imgs to get rid of the ones without proper OCR
def non_nan_str(val): return "" if isinstance(val, float) else val
# use updated if exist and then started_off
def final_answer(start, updated): 
    result = non_nan_str(updated)
    if result == "": return non_nan_str(start)
    else: return result

def get_qa_json(posts, course=COURSE):
    with open(f"{BASE}/{course}_QA_simplified_noOCR.json", 'w') as json_file:
        json_list = []
        for index, post in posts.iterrows():
            if post['num_images'] > 0: continue
            subject = post['subject']
            question = non_nan_str(post['question'])
            instruction = f"{subject}\n{question}"
            answer = final_answer(post['started_off_answer'], post['updated_answer'])
            # role = 'You are a helpful and knowledgeable teaching assistant in an introductory programming class on Matlab and C. Answer this question for a student.'
            # prompt = f"{role}\n### Question: {instruction} \n### Answer: {answer}"
            json_dict = {
                "id": len(json_list), #not unique: str(post["semester"])+ "_" + post["id_q"],
                "semester": post["semester"],
                "folder": post["folders"],
                "answer_by_instructor": post["answer_by"] == "instructor",
                "question": instruction,
                "answer": answer,
            }
            json_list.append(json_dict)
        json.dump(json_list, json_file, indent=4)
        print("Jsonify total #QA", len(json_list), "in course", course)
    return json_list

# if filter out the ones with images or answered by students
get_qa_json(all_qas)

Jsonify total #QA 2 in course sample_course


[{'id': 0,
  'semester': 'sample_semester',
  'folder': 'logistics',
  'answer_by_instructor': True,
  'question': 'coursebook unlock\nwhen i currently try to download the matlab coursebook, it says the file is locked. could this please be unlocked?',
  'answer': 'Seems to be unblocked now. Try again?'},
 {'id': 1,
  'semester': 'sample_semester',
  'folder': 'c_project',
  'answer_by_instructor': False,
  'question': 'Can you put an if statement inside another if statement?\n.',
  'answer': "Can you? Yes. Should you? Sometimes. For the project, it's perfectly okay.\n\nGenerally, it's considered bad practice as:\n\n\nif (A) {\n\tif (B) {\n\t\t// Some code\n\t}\n}\ncan always be written as:\n\nif (A && B) {\n\t// Some code\n}\n\nBut, there isn't really an actual issue with it outside of readability."}]

In [25]:
# len(all_qas[all_qas["num_images"] == 0])
# len(all_qas[all_qas["answer_by"] == "student"])
print(len(all_qas))
all_qas.columns

3


Index(['semester', 'postnum', 'subject', 'question', 'started_off_answer',
       'updated_answer', 'answer_by', 'folders', 'views', 'html_q', 'html_a',
       'id_q', 'id_a', 'utc_q', 'utc_a', 'numlikes_q', 'numlikes_a',
       'endorsed_q', 'endorsed_a', 'num_images'],
      dtype='object')

# Prepare data for RLHF 
```
[{
'instruction': student question,
'input': '',
'output1': started off answer,
'output2': final updated answer,
'preference': 2
}]
```

In [28]:
def get_rlanswer_json(posts, course=COURSE):
    with open(f"{BASE}/{course}_RL_answers_id.json", 'w') as json_file:
        json_list = []
        for index, post in posts.iterrows():
            # uncomment the next line to filter out the posts with images
            # if post['num_images'] > 0: continue
            subject = post['subject']
            question = non_nan_str(post['question'])
            instruction = f"{subject}\n{question}"
            updated_answer = non_nan_str(post['updated_answer'])
            if non_nan_str(updated_answer) == "":
                continue # skip the ones without updated answer
            assert updated_answer != "", "updated_answer is empty"  
            json_dict = {
                "semester": post["semester"],
                "id_q": post["id_q"],
                "instruction": instruction,
                "input": '',
                "output1": non_nan_str(post['started_off_answer']),
                "output2": updated_answer,
                "preference": 2
            }
            json_list.append(json_dict)
        json.dump(json_list, json_file, indent=4)
            
    print("Jsonify total started & updated answer", len(json_list), "in course", course)
    # return json_list

# total started & updated answer
get_rlanswer_json(all_qas)

Jsonify total started & updated answer 1 in course sample_course


# Recreate thread & followups from the data
- using `class_content_flat.json` fields such as
  - 'id': 'l75cspcyk4f1xb',
  - 'content': ```<p></p>```,
    - look up content from `contributions.csv` -> `Submission` column if there's exact match
  - 'created': '2022-08-22T22:52:27.874Z',
    - look up content from `contributions.csv` -> `Created At` column if there's exact match (e.g.,'2021-12-14 16:08:43 UTC')
  - 'parent_id': 'l7272ltvzwk53o',
  - 'thread_id': 'l6zorg9l2ky5mn',

### Followups
- note that course may have those FAQ note posts that many Q&A will exist in followups
- {COURSE}_{SEMESTER}_followups.csv
- **TODO**: a possible way to do it is to merge adjacent student posts & instructor posts after sorted by time

### Fields:
- postnum: 
- follownum: first sort followup of a note/question and append to postnum like 590_f1, etc.
- utc: can be used to sort & merge adjacent posts together
- thread_type: note/question [can use postnum to access the corresponding note/qa text]
- type: followup/followup_response
- by: student/instructor
- content: 

In [35]:
# make a df from all_posts with columns
def get_followups_stats(all_posts, all_instructors):
    followups_stats = []
    for postid, posts in all_posts.items():
        semester, postnum = "_".join(postid.split('_')[:-1]), postid.split('_')[-1]
        instructors = all_instructors[semester]
        num_followup_and_response, len_followup_and_response_text, num_followup_and_response_images, \
        num_by_student, num_by_instructor, len_by_student, len_by_instructor, img_by_student, img_by_instructor = 0, 0, 0, 0, 0, 0, 0, 0, 0
        posttype = []
        has_followup = False
        for post in posts:
            folders = post['folders']
            if post['type'] in ['followup', 'followup_response']:
                has_followup = True
                num_followup_and_response += 1
                len_post, img_post = len(post['clean_content']), post['content'].count("<img")
                len_followup_and_response_text += len_post
                num_followup_and_response_images += img_post
                if post["editors"][0] in instructors:
                    num_by_instructor += 1
                    len_by_instructor += len_post
                    img_by_instructor += img_post
                else: 
                    num_by_student += 1
                    len_by_student += len_post
                    img_by_student += img_post
            else: posttype.append(post['type'])
        if has_followup: 
            d = {'semester': semester, 'postnum': postnum, 'posttype': posttype, 'folders': folders}
            s = {**d, 'type': 'student', 'num_followup_and_response': num_by_student, 'len_followup_and_response_text': len_by_student, 'num_followup_and_response_images': img_by_student}
            followups_stats.append(s)
            i = {**d, 'type': 'instructor', 'num_followup_and_response': num_by_instructor, 'len_followup_and_response_text': len_by_instructor, 'num_followup_and_response_images': img_by_instructor}
            followups_stats.append(i)
    return pd.DataFrame(followups_stats)

followups_stats = get_followups_stats(all_posts, all_instructors)
followups_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 8 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   semester                          4 non-null      object
 1   postnum                           4 non-null      object
 2   posttype                          4 non-null      object
 3   folders                           4 non-null      object
 4   type                              4 non-null      object
 5   num_followup_and_response         4 non-null      int64 
 6   len_followup_and_response_text    4 non-null      int64 
 7   num_followup_and_response_images  4 non-null      int64 
dtypes: int64(3), object(5)
memory usage: 384.0+ bytes


In [37]:
followups_stats

,semester,postnum,posttype,folders,type,num_followup_and_response,len_followup_and_response_text,num_followup_and_response_images
0,sample_semester,7,"[question, i_answer]",logistics,student,0,0,0
1,sample_semester,7,"[question, i_answer]",logistics,instructor,1,44,0
2,sample_semester,1477,"[question, s_answer]",c_project,student,2,47,0
3,sample_semester,1477,"[question, s_answer]",c_project,instructor,2,585,0


### Ad-hoc data analysis

In [40]:
print("all_qas", len(all_qas), all_qas.columns)
print("all_notes", len(all_notes), all_notes.columns)
print("all_posts", type(all_posts), len(all_posts))

all_qas 3 Index(['semester', 'postnum', 'subject', 'question', 'started_off_answer',
       'updated_answer', 'answer_by', 'folders', 'views', 'html_q', 'html_a',
       'id_q', 'id_a', 'utc_q', 'utc_a', 'numlikes_q', 'numlikes_a',
       'endorsed_q', 'endorsed_a', 'num_images'],
      dtype='object')
all_notes 0 Index(['semester', 'postnum', 'id', 'subject', 'utc', 'note', 'html'], dtype='object')
all_posts <class 'dict'> 3


In [41]:
all_posts_df = pd.DataFrame([{"semester": postid.split("_")[0], **post} for postid, posts in all_posts.items() for post in posts])
all_posts_df.groupby(['type']).agg({'endorsed': 'sum', 'views': 'sum', 'score': 'sum', 'postnum': 'count'})

# split the folders column into multiple rows by ";"
def split_folders(folders):
    if isinstance(folders, float) or folders is None: return []
    else: return folders.split("; ")

all_posts_df['folders'] = all_posts_df['folders'].apply(split_folders)
all_posts_df = all_posts_df.explode('folders')
all_posts_df.groupby(['semester', 'folders']).agg({'endorsed': 'sum', 'views': 'sum', 'score': 'sum', 'postnum': 'count'})

endorsed  views  score  postnum
semester folders                                   
sample   c_project         0  114.0    0.0        6
         lab5              0  281.0    0.0        2
         logistics         1   12.0    0.0        3